# **Project Name**    - Rosamann Retails Sales Prediction



##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Member 1 -**  - Jayesh Prakash Dahiwale


# **Project Summary -**

**Rossmann operates over 3,000 drug stores in 7 european countries. So our porject aim is to predict their daily sales for upto six weeks in advance. Stores sales are influenced by many factors, including promotions, competitions,school and state holidays, seasonality and locality. So In this project we have been provided the data from 1,500 Rossmann stores. So our task is to forecast the "Sales" column for the test** 
##<p>Provided there are two datasets, one is **Store** dataset having <b>1,115</b> observations in it with <b>10</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>
## <p>Other datatset is about **Sales** dataset having <b>1,017,209</b> observations in it with <b>9</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>

# **GitHub Link -**

https://github.com/Jayeshdahiwale/RegressionCapstoneProjectII

# **DataSet Link -**


Dataset DriveLink:https://drive.google.com/drive/folders/1XtTzgmtM-e9jGHbIq8pLzIbo3C8NF1T2?usp=share_link



## **Questions for analysis are as follows**
**1) Visualising the distribution of "Sales" & "Customers"?**<br>
**2) Statistics of Sales column ?** <br>
**3) Which rows are unnecessary and need to be removed ??** <br>
**4) What are the outliers ?** <br>
**5) Establishing relationship between Sales and Customers ?** <br>
**6) How stores are performing in Sales by month based on Assortment type ?**<br>
**7) How UPT metric compares across stores of different assortment types ?**<br>
**8)Correlation between competition distance and UPT metric?**<br>
**9)Which linear regression model is best?**<br>




#### **Define Your Business Objective?**

Increase the number of sales by predicting the rates at at optimal rate and finding the best suitalbe condition which attract the customers thereby increasing the profit for the Drug Store.

## **Importing the required packages**

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import math

We'll begin with the required libraries and reading our .csv files

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Reading the dataset file
# dataset file googgle drive link : https://drive.google.com/file/d/185htr6OyxAZ0xb9QwxHpsUn8eTlq58v5/view?usp=share_link
working_directory_path = '/content/drive/MyDrive/MachineLearningAlmabetterJourney/CapstoneProject2/'



In [ ]:
store_lookup = pd.read_csv(working_directory_path + 'store.csv')

In [ ]:
sales_data = pd.read_csv(working_directory_path + 'Rossmann Stores Data.csv')


In [ ]:
store_lookup.head()

In [ ]:
store_lookup.shape

In [ ]:
store_lookup.info()

In [ ]:
def null_cols(data):
  columns = data.columns
  null_cols = [(col,data[col].isna().sum()) for col in columns if data[col].isna().sum()!=0]
  return null_cols

In [ ]:
null_cols(store_lookup)

We can see in store lookup table there are total 6 null columns

In [ ]:
# Lets see the sales data
sales_data.head()

In [ ]:
sales_data.shape

In [ ]:
sales_data.info()

These is no null columns in this dataset

In [ ]:
#lets see the starting date and end date of sales data
print(sales_data['Date'].min())
print(sales_data['Date'].max())

**Now we will merge the two data resources so we can more easily work with them. We will join the tables baes on the shared store column, which is a foreign key in the sales_data table and primary key in the stores_llokup table, so we'll validate the merge baed on this many to one relationship**

In [ ]:
merged_sales = sales_data.merge(store_lookup,how='left',on='Store',validate = 'many_to_one')

In [ ]:
merged_sales.info()

In [ ]:
merged_sales.shape

In [ ]:
merged_sales.head()

In [ ]:
merged_sales.describe()

In [ ]:
merged_sales.info()

**Lets check the skewness of columns i.e.** <br>
 "Day of Week","Sales","Customers","Competion Distance"

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].skew()

we can see here competition distance,customers are positively skewed<br>


# **<p>Lets check the distribution bell peak using kurotsis</p>**

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].kurtosis()

We can see the that competition distance,and customers has highest peak...And sales and Day of week has negative kurtosis

# **Question 1: Visualising the distribution of Sales and Customer columns**

# **<p>Lets visualize the skewness and kurtosis</p>**

In [ ]:
fig,axes = plt.subplots(1,2)
sns.distplot(merged_sales['Sales'],color="y",ax=axes[0])
sns.distplot(merged_sales['Customers'],color="y",ax=axes[1])
fig.set_size_inches(15,5)

Histograms of our Sales and Customers values show us a positive skew and high kurtosis.

# **Question 2: Statistics of Sales column**

# <p>**Next we will take a closer look at our statistics for sales column.**</p>

In [ ]:
print("skew     " + str(round(merged_sales['Sales'].skew(),6)))
print("kurtosis " + str(round(merged_sales['Sales'].kurtosis(),6)))
print(merged_sales['Sales'].describe().round(3))
print("mode     " + str(merged_sales['Sales'].mode()))

We can see our sales figures have a slightly positive skew, with the **mean (5773.819)** only slightly larger than the **median (5744.000)**, suggesting most outliers are to the right of the mean.

High kurtosis indicates it's **leptokurtic** with the likelihood of heavy tails and outliers that may be extreme. Considering our min and max values of **0** and **41,551** sales, we aren't surprised to see there may be some extreme outliers.

The max value well above the mean of 5,773.819 and outside the **standard deviation of 3849.926** helps us see how our mean ends up getting pulled slightly to the right for our positive skew.

There is no mode as we don't have any stores recording the exact same number of sales on any days, which isn't surprising.